Import Required Packages

In [1]:
import duckdb
import pandas as pd

Check available tables

In [2]:
sql_query = '''
show tables
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,name
0,bronze_parking_violation_codes
1,bronze_parking_violations
2,first_model
3,gold_ticket_metrics
4,gold_vehicles_metrics
5,parking_violation_codes
6,parking_violations_2023
7,ref_model
8,silver_parking_violation_codes
9,silver_parking_violations


Create tables

In [4]:
sql_query_import_data_1 = '''
CREATE OR REPLACE TABLE parking_violation_codes AS
SELECT * FROM read_csv_auto(
    'data/dof_parking_violation_codes.csv',
     normalize_names=True
);
'''

sql_query_import_data_2 = '''
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT * FROM read_csv_auto(
    'data/parking_violations_issued_fiscal_year_2023_sample.csv',
     normalize_names=True
);
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    con.sql(sql_query_import_data_1)
    con.sql(sql_query_import_data_2)

Check the tables are created

In [5]:
sql_query = '''
show tables
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,name
0,first_model
1,parking_violation_codes
2,parking_violations_2023
3,ref_model


Print first 5 rows in the parking_violation_codes & parking_violations_2023 

In [6]:
with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql('''SELECT * FROM parking_violation_codes LIMIT 5;'''))
    display(con.sql('''SELECT * FROM parking_violations_2023 LIMIT 5;'''))

┌───────┬────────────────────────────────┬─────────────────────────┬─────────────────┐
│ code  │           definition           │ manhattan_96th_st_below │ all_other_areas │
│ int64 │            varchar             │          int64          │      int64      │
├───────┼────────────────────────────────┼─────────────────────────┼─────────────────┤
│     1 │ FAILURE TO DISPLAY BUS PERMIT  │                     515 │             515 │
│     2 │ NO OPERATOR NAM/ADD/PH DISPLAY │                     515 │             515 │
│     3 │ UNAUTHORIZED PASSENGER PICK-UP │                     515 │             515 │
│     4 │ BUS PARKING IN LOWER MANHATTAN │                     115 │             115 │
│     5 │ BUS LANE VIOLATION             │                     250 │             250 │
└───────┴────────────────────────────────┴─────────────────────────┴─────────────────┘

┌────────────────┬────────────────────┬────────────┬───┬───────────────────┬──────────────────────┐
│ summons_number │ registration_state │ plate_type │ … │ hydrant_violation │ double_parking_vio…  │
│     int64      │      varchar       │  varchar   │   │      varchar      │       varchar        │
├────────────────┼────────────────────┼────────────┼───┼───────────────────┼──────────────────────┤
│     9010912681 │ CA                 │ PAS        │ … │ NULL              │ NULL                 │
│     4858762841 │ NY                 │ PAS        │ … │ NULL              │ NULL                 │
│     4854645684 │ FL                 │ PAS        │ … │ NULL              │ NULL                 │
│     9044582707 │ NY                 │ PAS        │ … │ NULL              │ NULL                 │
│     9041503330 │ NY                 │ PAS        │ … │ NULL              │ NULL                 │
├────────────────┴────────────────────┴────────────┴───┴───────────────────┴──────────────────────┤


In [7]:
sql_query = '''
show tables
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,name
0,first_model
1,parking_violation_codes
2,parking_violations_2023
3,ref_model


In [8]:
with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql('''SELECT * FROM parking_violations_2023 LIMIT 5;''').df().columns.tolist())

['summons_number',
 'registration_state',
 'plate_type',
 'issue_date',
 'violation_code',
 'vehicle_body_type',
 'vehicle_make',
 'issuing_agency',
 'vehicle_expiration_date',
 'violation_location',
 'violation_precinct',
 'issuer_precinct',
 'issuer_code',
 'issuer_command',
 'issuer_squad',
 'violation_time',
 'time_first_observed',
 'violation_county',
 'violation_in_front_of_or_opposite',
 'date_first_observed',
 'law_section',
 'sub_division',
 'violation_legal_code',
 'days_parking_in_effect',
 'from_hours_in_effect',
 'to_hours_in_effect',
 'vehicle_color',
 'unregistered_vehicle',
 'vehicle_year',
 'meter_number',
 'feet_from_curb',
 'no_standing_or_stopping_violation',
 'hydrant_violation',
 'double_parking_violation']

In [9]:
with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql('''SELECT * FROM bronze_parking_violations LIMIT 5;''').df())

,summons_number,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,vehicle_expiration_date,violation_location,violation_precinct,issuer_precinct,issuer_code,issuer_command,issuer_squad,violation_time,violation_county,violation_legal_code,vehicle_color,vehicle_year
0,9010912681,CA,PAS,2022-10-11,17,SUBN,FORD,T,20220788,50.0,50,50,365910,T201,D,0825A,BX,NaN,BLACK,0
1,4858762841,NY,PAS,2023-08-21,36,4DSD,HONDA,V,0,NaN,0,0,0,None,None,1036A,BK,True,GY,2003
2,4854645684,FL,PAS,2023-07-26,36,UT,BMW,V,0,NaN,0,0,0,None,None,0126P,QN,True,WHI,2022
3,9044582707,NY,PAS,2023-04-10,21,SUBN,SUBAR,T,20231217,79.0,79,79,370326,T301,H,0924A,K,NaN,GY,2017
4,9041503330,NY,PAS,2023-03-21,21,4DSD,CHEVR,T,20250320,26.0,26,26,373097,T103,C,1155A,NY,NaN,BK,2018


In [10]:
with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql('''SELECT COUNT(*) AS COUNT FROM silver_parking_violation_codes;''').df())

,COUNT
0,194


In [11]:
with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql('''SELECT COUNT(*) AS COUNT FROM parking_violation_codes;''').df())

,COUNT
0,97


In [4]:
sql_query_import_data_1 = '''
CREATE OR REPLACE TABLE parking_violation_codes AS
SELECT * FROM read_csv_auto(
    'data/dof_parking_violation_codes.csv',
     normalize_names=True
);
'''

sql_query_import_data_2 = '''
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT * FROM read_csv_auto(
    'data/parking_violations_issued_fiscal_year_2023_sample.csv',
     normalize_names=True
);
'''

with duckdb.connect('data/prod_nyc_parking_violations.db') as con:
    con.sql(sql_query_import_data_1)
    con.sql(sql_query_import_data_2)